# Time-Synchronized Recordings

## Goals

* Learn how to validate VRS files recorded in a [TICSync](https://facebookresearch.github.io/projectaria_tools/docs/ARK/sdk/ticsync) session.
* Learn how fetch synchronized frames from TICSync VRS files.
* Learn how to interpret synchronization offsets amongst the frames of the recordings. 

## Understanding Time Domains

In a TICSync recording, all devices mark video frames with a timestamp in a conceptual TICSync time domain. During the recording, the TICSync algorithm constructs, on-the-fly, the mapping between the conceptual TICSync time domain and the concrete `DEVICE_TIME` time domains of the glasses. Under the current implementation, the unique _server_ device uses its `DEVICE_TIME` as the conceptual TICSync time, while all clients use their concrete `TIC_SYNC` time domains. The code below shows how to download TICSync sample recordings (VRS files) and how to query the concrete time domains in the VRS files.

## Python Dependencies

1. Set up a Python virtual environment with [this version of Projectaria Tools using pip](https://facebookresearch.github.io/projectaria_tools/docs/data_utilities/installation/installation_python)
2. You may have to `pip install matplotlib notebook==6.5.7`. Notebook v7 may have issues.
4. `cd ~ && jupyter notebook`.
5. Navigate in jupyter's file browser to the location of this notebook

Alternatively, try the notebook in colab.

## Download Sample TICSync VRS Files

We have three sample synchronized recordings of the same millisecond-resolution clock display. The recordings include 3 minutes of simultaneous recording. Some of the recordings are longer than three minutes long because the glasses take time to connect and they start recording sequentially.

The files are around 3&nbsp;GB each, so the downloading may take some considerable time. Check your `/tmp/ticsync_sample_data` folder to track download progress. The notebook kernel may appear frozen during the downloads, but it's not. The cell below will finish eventually. 

The logic checks whether the files have already been downloaded so you only have to wait once, then you can repeatedly run the notebook.

If you prefer, you may substitute your own TICSync files for our samples. Just bypass this downloading code (don't run the cells) and adjust the definition of `ticsync_pathnames` as needed.

In [ ]:
import os
from tqdm import tqdm
from urllib.request import urlretrieve
from zipfile import ZipFile

google_colab_env = 'google.colab' in str(get_ipython())
if google_colab_env:
    print("Running from Google Colab, installing projectaria_tools and getting sample data")
    !pip install projectaria-tools
    ticsync_sample_path = "./ticsync_sample_data/"
else:
    ticsync_sample_path = "/tmp/ticsync_sample_data/"

base_url = "https://www.projectaria.com/async/sample/download/?bucket=core&filename="
os.makedirs(ticsync_sample_path, exist_ok=True)

ticsync_filenames = [
    "ticsync_tutorial_server_3m.vrs",
    "ticsync_tutorial_client1_3m.vrs",
    "ticsync_tutorial_client2_3m.vrs",]

print("Downloading sample data (if they don't already exist)")
for filename in tqdm(ticsync_filenames):
    print(f"Processing: {filename}")
    full_path = os.path.join(ticsync_sample_path, filename)
    if os.path.isfile(full_path):
        print(f"{full_path} has already been downloaded.")
    else:
        print(f"Downloading {base_url}{filename} to {full_path}")
        urlretrieve(f"{base_url}{filename}", full_path)
        if filename.endswith(".zip"):
            with ZipFile(full_path, 'r') as zip_ref:
                zip_ref.extractall(path=ticsync_sample_path)

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Iterator, Any
import rerun as rr

from projectaria_tools.core import data_provider
from projectaria_tools.core.data_provider import VrsMetadata, MetadataTimeSyncMode
from projectaria_tools.core.sensor_data import (
    SensorData,
    ImageData,
    TimeDomain,
    TimeQueryOptions,
)
from projectaria_tools.core.stream_id import StreamId

## Pathname Instructions

Adjust the following path names if necessary to accommodate the locations of the files you wish to analyze. These are the only names needed going forward.

In [ ]:
ticsync_pathnames = [
    os.path.join(ticsync_sample_path, filename)
    for filename in ticsync_filenames]

## Get Data Providers

In [ ]:
data_providers = [data_provider.create_vrs_data_provider(filename)
                  for filename in ticsync_pathnames]

## Get and Browse Metadata

Let's examine the metadata for one of the providers.

The metadata are in a Python object. Here is a way to convert it (or any other object) into a dict for browsing its fields.

In [ ]:
server_metadata: VrsMetadata = data_providers[0].get_metadata()
server_metadata

The metadata have the attribute `time_sync_mode`, which is a Python `enum`. We use its `name` attribute for tests below.

In [ ]:
str(MetadataTimeSyncMode.TicSyncServer)

In [ ]:
server_metadata.time_sync_mode.name

The other possibilities for `MetadataTimeSyncMode` are `TicSyncClient`, `Ntp`, `Timecode`, and `NotEnabled`. Only `TicSyncServer` and `TicSyncClient` pertain to our investigations, here.

## Check that all VRS Files Belong to the Same Session

One critical attribute of the metadata is the `shared_session_id`. Your shared recordings must belong to the same shared session. If they do not, the results are nonsense.

In [ ]:
def print_session_ids(providers:list[data_provider]) -> None:
    for provider in providers:
        print(f'shared session id = {provider.get_metadata().shared_session_id}')

def check_session_ids(providers:list[data_provider]) -> None:
    session_ids = [provider.get_metadata().shared_session_id
                  for provider in providers]
    assert (sid == session_ids[0] for sid in session_ids)

In [ ]:
print_session_ids(data_providers)
check_session_ids(data_providers)

We'll use `check_session_ids` in the display codes below.

## Displaying Frames by Timestamp(ns)

First define a `streams` dictionary, which reminds us that there are other synchronized recordings in the VRS files. We use only `"camera-rgb"` in this notebook.

After these definitions, we'll see how to investigate the synchronized VRS files by timestamp.

In [ ]:
# It's possible to search other image streams.
streams = {
    "camera-slam-left": StreamId("1201-1"),
    "camera-slam-right":StreamId("1201-2"),
    "camera-rgb":StreamId("214-1"),
    "camera-eyetracking":StreamId("211-1"),}

In [ ]:
def get_server_provider(providers:list[data_provider]) -> data_provider:
    server_providers = [provider for provider in providers
                       if provider.get_metadata().time_sync_mode.name == 'TicSyncServer']
    return server_providers[0]

See that it's a valid Python object.

In [ ]:
server_provider = get_server_provider(data_providers)

Get all the timestamps, in nanoseconds, from the reference `DEVICE_TIME` time domain of the server. These are the conceptual TICSync timestamps for the unique, distinguished server device.

In [ ]:
all_server_timestamps_ns = server_provider.get_timestamps_ns(
    streams["camera-rgb"], TimeDomain.DEVICE_TIME)

### Helper Functions

In [ ]:
MS_PER_NS = 1 / 1_000_000

def ticsync_time_domain_from_provider(provider:data_provider) -> TimeDomain:
    """Return a VRS file's local approximation of the conceptual
    TICSync time."""
    mode = provider.get_metadata().time_sync_mode.name
    if mode == 'TicSyncServer':
        domain = TimeDomain.DEVICE_TIME
    elif mode == 'TicSyncClient':
        domain = TimeDomain.TIC_SYNC
    else:
        raise NotImplementedError(f'Unsupported time-sync mode {mode}')
    return domain

def split_providers(providers:list[data_provider]) -> tuple[data_provider, list[data_provider]]:
    """A utility function used internally."""
    server_provider = [provider for provider in providers
                       if provider.get_metadata().time_sync_mode.name
                          == 'TicSyncServer'][0]
    client_providers = [provider for provider in providers
                       if provider.get_metadata().time_sync_mode.name
                          == 'TicSyncClient']
    return server_provider, client_providers

def print_timestamp_offsets_ms(time_ns:int, providers:list[data_provider]) -> None:
    """We are concerned with the offsets (time differences) between
    client glasses and server glasses. Offsets between clients are
    not informative, as each client settles to an approximation of
    the server's timestamps."""
    server_provider, client_providers = split_providers(providers)
    server_time_ns = get_closest_timestamp_ns(time_ns, server_provider)
    client_times_ns = [get_closest_timestamp_ns(time_ns, client_provider)
                       for client_provider in client_providers]
    for i, client_time_ns in enumerate(client_times_ns):
        offset = (client_time_ns - server_time_ns) * MS_PER_NS
        print(f'client{i + 1} offset (ms) = {offset}')

def get_closest_timestamp_ns(ticsync_time_ns:int, provider:data_provider) -> int:
    """Return the actual timestamp in a VRS file that's closest
    to a given time in nanoseconds."""
    domain = ticsync_time_domain_from_provider(provider)
    return provider.get_sensor_data_by_time_ns(
        stream_id=streams["camera-rgb"],
        time_ns=ticsync_time_ns,
        time_domain=domain,
        time_query_options=TimeQueryOptions.CLOSEST).get_time_ns(domain)

def get_closest_image_by_ticsync_time(ticsync_time_ns:int, provider:data_provider) -> ImageData:
    """Get an image from a VRS file closest in TICSync time to a
    given time in nanoseconds."""
    return provider.get_image_data_by_time_ns(
        stream_id=streams["camera-rgb"],
        time_ns=ticsync_time_ns,
        time_domain=ticsync_time_domain_from_provider(provider),
        time_query_options=TimeQueryOptions.CLOSEST)

### Show Frames by Timestamp

In [ ]:
def show_frames_by_ticsync_timestamp_ns(ticsync_time_ns:int, providers:list[data_provider]) -> None:
    check_session_ids(providers)
    images = [get_closest_image_by_ticsync_time(ticsync_time_ns, provider)
             for provider in providers]
    print_timestamp_offsets_ms(ticsync_time_ns, providers)
    fig_m, axes_m = plt.subplots(1, len(providers), figsize=(10, 5), dpi=300)
    image_index = 0
    for idx, frame in enumerate(images):
        axes_m[idx].set_title(providers[idx].get_metadata().time_sync_mode.name)
        npa = frame[0].to_numpy_array()
        npar = np.rot90(npa, k=1, axes=(1, 0))
        axes_m[idx].imshow(npar)
    plt.show()

### At an Arbitrary Timestamp

Arbitrarily, pick a timestamp halfway through the recording.

In [ ]:
show_frames_by_ticsync_timestamp_ns(all_server_timestamps_ns[len(all_server_timestamps_ns) // 2], data_providers)

See that they're synchronized to clock time within 16 ms, within one frame of each other.

## Waiting for TICSync Settling

TICSync needs warmup, typically 45 seconds after recording starts for each device to settle. Here is code to show you how to find timestamps after this settling time.

In [ ]:
SEC_PER_NS = 1 / 1e9

def diff_timestamps_ns_s(t1_ns:int, t2_ns:int) -> int:
    return (t1_ns - t2_ns) * SEC_PER_NS

def timestamp_ns_after_delay_s(timestamps_ns:list[int], delay_s:int) -> int:
    first_timestamp_ns = timestamps_ns[0]
    ts_ns = 0
    for i, ts_ns in enumerate(timestamps_ns):
        if diff_timestamps_ns_s(ts_ns, first_timestamp_ns) >= delay_s:
            break
    return ts_ns

The TICSync time after 45 seconds since last device began recording.

In [ ]:
ticsync_time_ns_after_settlement = max([timestamp_ns_after_delay_s(
    provider.get_timestamps_ns(
        streams["camera-rgb"],
        ticsync_time_domain_from_provider(provider)), 45)
    for provider in data_providers])

That allows us to display the first frames after the delay.

In [ ]:
show_frames_by_ticsync_timestamp_ns(ticsync_time_ns_after_settlement, data_providers)

Observe, again, that the offsets are within one frame.

## Play Videos in Rerun

Rerun is an open-source tool for displaying and manipulating visualizations with interactive tools. The `scrubbing` tool is particularly effective for exploring TICSync timestamps. 

Rerun can display in an external window with frames fed on-the-fly from the notebook, or Rerun can display directly in the notebook after waiting for all frames of interest to be logged. In this notebook, we demonstrate the displaying of frames in the notebook itself, and we present non-executable instructions for those who would like to display frames in an external window.

### Iterate over all Server Frames

Both modes of Rerun--in-notebook and in-external-window--employ a Python iterator over all the server's frames to retrieve frames marked with the TICSync reference timestamps.

In [ ]:
def create_reference_iterator(server_provider:data_provider) -> Iterator[SensorData]:
    # Set up deliver options for the server VRS data provider.
    deliver_option = server_provider.get_default_deliver_queued_options()
    # Deactivate all server streams, for a fresh start.
    deliver_option.deactivate_stream_all()
    # Activate the one camera-rgb stream we care about.
    deliver_option.activate_stream(streams["camera-rgb"])
    # Create a timestamp-ordered, sensor-data iterator from the server.
    # This will work for large streams like the ~4GB VRS files for the
    # three-minute videos.
    result: Iterator = server_provider.deliver_queued_sensor_data(deliver_option)
    return result

### Starting Rerun

Before logging data, it is necessary to start Rerun. The code differs depending on whether you're running Rerun in an external window. If so, change the type of the following cell to `Code` via the `Cell->Cell Type` menu in Jupyter and run it. For colab users and others who prefer an in-notebook display, this code is in a cell of type `Raw`.

To start Rerun for in-notebook display, run the following cell. Run the identical code every time you want a fresh Rerun display in the notebook. Note that Rerun may not display if you're running the notebook in PyCharm. Best to use either colab or the web-based implementation of Jupyter.

In [ ]:
rr.init("Aria TICSync Visualizer")

### log_datum

`log_datum`, called by `show_rerun`, pushes images into Rerun.

In [ ]:
def log_datum(
    is_server: bool,
    device_nickname: str,  # to ID a device Rerun's display
    sensor_name: str,  # e.g., 214-1 for RGB camera
    label: str,  # e.g., camera-rgb; for double-checking
    datum: SensorData,
    timestamp_ns: int,
) -> None:
    """Called by show_rerun."""
    # Set device_entity for labeling displays in Rerun.
    rr.set_time_nanos("device_time", timestamp_ns)
    if is_server:
        device_entity = "/server/" + device_nickname + "/"
    else:
        device_entity = "/client/" + device_nickname + "/"

    rr_stream_name = device_entity + sensor_name
    assert "camera" in label
    # The datum has its image data in slot 0.
    image_index = 0
    img = datum.image_data_and_record()[image_index].to_numpy_array()
    # Rotate the image for readability of the clock.
    rotated_img = np.rot90(img, k=1, axes=(1, 0))
    rr.log(
        rr_stream_name + "/image",
        rr.Image(rotated_img).compress(jpeg_quality=75),)

### show_rerun

`show_rerun` fetches client frames via `get_sensor_data_by_time_ns` in the client's `TIC_SYNC` time domain. Alternatively, one might use `get_closest_image` as we did above.

#### Nicknames for Rerun Display

The last six digits of device serial numbers are a convenient choice for "nicknames" for the glasses in the Rerun display. However, nicknames can be anything you like. Change the code below to suit you.

`show_rerun` takes a list of providers, a starting timestamp in ns, and a number of frames, defaulting to 300--tens seconds worth at 30 frames per second. If you are running in-notebook, there can be a delay as the iterator moves to the desired timestamp from the beginning of the server file. 

In [ ]:
def show_rerun(providers: list[data_provider], start_ticsync_timestamp_ns:int, nframes:int = 300):
    check_session_ids(providers)
    assert nframes > 0
    server_provider, client_providers = split_providers(providers)
    server_data_stream = create_reference_iterator(server_provider)
    server_nick = server_provider.get_metadata().device_serial[-6:]
    client_nicks = [client_provider.get_metadata().device_serial[-6:]
                   for client_provider in client_providers]
    print(f'{server_nick = }')
    print(f'{client_nicks = }')
    startframe = 0
    for i, server_datum in tqdm(enumerate(server_data_stream)):
        # The server datum contains the server image. Get its timestamp,
        # in the reference TimeDomain DEVICE_TIME.
        server_timestamp_ns = server_datum.get_time_ns(TimeDomain.DEVICE_TIME)
        if server_timestamp_ns < start_ticsync_timestamp_ns:
            continue
        elif startframe == 0:
            startframe = i
        if (i - startframe) >= nframes:
            break
        # The rest of these attributes help with labeling the Rerun displays.
        stream_id = server_datum.stream_id()  # e.g. 214-1 for the RGB camera.
        stream_id_str = str(stream_id)  # as a string
        # The following will say "camera-rgb", for labeling the Rerun displays.
        stream_label = server_provider.get_label_from_stream_id(server_datum.stream_id())
        log_datum(
            is_server=True,
            device_nickname=server_nick,
            sensor_name=stream_id_str,
            label=stream_label,
            datum=server_datum,
            timestamp_ns=server_timestamp_ns,)
        for i, (nickname, provider) in enumerate(zip(client_nicks, client_providers)):
            # Get the closest client sensor datum, in its TIC_SYNC TimeDomain,
            # to the server's reference time in its DEVICE_TIME TimeDomain.
            client_datum = provider.get_sensor_data_by_time_ns(
                stream_id=stream_id,
                time_ns=server_timestamp_ns,
                time_domain=TimeDomain.TIC_SYNC,
                time_query_options=TimeQueryOptions.CLOSEST,
            )
            # Key the Rerun display of the client image to the client's TIC_SYNC time.
            client_timestamp_ns = client_datum.get_time_ns(TimeDomain.TIC_SYNC)
            # Log the client image.
            log_datum(
                is_server=False,
                device_nickname=nickname,
                sensor_name=stream_id_str,
                label=stream_label,
                datum=client_datum,
                timestamp_ns=client_timestamp_ns,)

### Show Three Videos

In the Rerun display, pan the images by dragging or two-finger scrolling on a trackpad, and zoom the images via pinching on a mac trackpad. Move the scrubbing tool, which appears below the images, to read off TICSync times in hours, minutes, and seconds. These times do not correspond to the times displayed on the real-time clock in the images. 

The following cell demonstrates 150 frames starting after a quarter of the entire duration. The cell takes about 20 seconds to run. 

In [ ]:
rr.init("Aria TICSync Visualizer")
start_time_ns = all_server_timestamps_ns[len(all_server_timestamps_ns) // 4]
show_rerun(data_providers, start_time_ns, nframes=150)
rr.notebook_show()

## Find Frames by Scrubbing in Rerun

Use Rerun's scrubbing feature to read off frame times in hours, minutes, and seconds. Display synchronized triplets of frames given timestamps in hours, minutes, seconds.

In [ ]:
def show_frames_by_h_m_s(hours:int, minutes:int, seconds:int):
    NS_PER_S = 1_000_000_000
    S_PER_M = 60
    S_PER_H = 60 * S_PER_M
    time_ns = NS_PER_S * ((hours * S_PER_H) + (minutes * S_PER_M) + seconds)
    show_frames_by_ticsync_timestamp_ns(time_ns, data_providers)

show_frames_by_h_m_s(9, 22, 54)

## Conclusion

Generally speaking, TICSync performs within 1 frame after a 45-second settling time.

We have exhibited general tools for displaying and manipulating synchronized data from VRS files. We have shown how to assess the synchronization versus a physical time standard such as a millisecond clock display. 